-- =========================
-- ===== SILVER LAYER =====
-- =========================

In [0]:
%sql
-- Set default schema for 2-part names
USE SCHEMA sales_sv;

Customers: typing, trimming, normalize empty email to NULL, deduplicate by customer_id (keep latest signup_date)

In [0]:
%sql
CREATE OR REPLACE TABLE sales_sv.customers_sv AS
SELECT
  CAST(customer_id AS BIGINT)                    AS customer_id,
  TRIM(first_name)                               AS first_name,
  TRIM(last_name)                                AS last_name,
  LOWER(NULLIF(email, ''))                       AS email,
  CAST(signup_date AS DATE)                      AS signup_date
FROM sales_bz.customers_bz
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY signup_date DESC) = 1;

Orders: typing, uppercase status, drop NULL amounts, deduplicate by order_id (keep latest order_ts)

In [0]:
%sql
CREATE OR REPLACE TABLE sales_sv.orders_sv AS
SELECT
  CAST(order_id AS BIGINT)                       AS order_id,
  CAST(customer_id AS BIGINT)                    AS customer_id,
  CAST(order_date AS TIMESTAMP)                  AS order_ts,
  UPPER(status)                                  AS status,
  CAST(amount AS DECIMAL(18,2))                  AS amount
FROM sales_bz.orders_bz
WHERE amount IS NOT NULL
QUALIFY ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY order_ts DESC) = 1;

Categories: basic typing/trim

In [0]:
%sql
CREATE OR REPLACE TABLE sales_sv.categories_sv AS
SELECT
  CAST(category_id AS BIGINT)                    AS category_id,
  TRIM(category_name)                            AS category_name,
  CAST(parent_category_id AS BIGINT)             AS parent_category_id
FROM common_bz.categories_bz;

Products: typing and trimming

In [0]:
%sql
CREATE OR REPLACE TABLE sales_sv.products_sv AS
SELECT
  CAST(product_id AS BIGINT)                     AS product_id,
  TRIM(product_name)                             AS product_name,
  CAST(category_id AS BIGINT)                    AS category_id,
  CAST(unit_price AS DECIMAL(18,2))              AS unit_price,
  CAST(active_flag AS INT)                       AS active_flag,
  CAST(created_at AS TIMESTAMP)                  AS created_at
FROM common_bz.products_bz;

Order Items: typing

In [0]:
%sql
CREATE OR REPLACE TABLE sales_sv.order_items_sv AS
SELECT
  CAST(order_item_id AS BIGINT)                  AS order_item_id,
  CAST(order_id AS BIGINT)                       AS order_id,
  CAST(product_id AS BIGINT)                     AS product_id,
  CAST(quantity AS INT)                          AS quantity,
  CAST(unit_price AS DECIMAL(18,2))              AS unit_price,
  CAST(line_amount AS DECIMAL(18,2))             AS line_amount
FROM sales_bz.order_items_bz;

Payments: standardize method/status + typing

In [0]:
%sql
CREATE OR REPLACE TABLE sales_sv.payments_sv AS
SELECT
  CAST(payment_id AS BIGINT)                     AS payment_id,
  CAST(order_id AS BIGINT)                       AS order_id,
  UPPER(TRIM(payment_method))                    AS payment_method,
  UPPER(TRIM(payment_status))                    AS payment_status,
  CAST(payment_amount AS DECIMAL(18,2))          AS payment_amount,
  CAST(payment_date AS TIMESTAMP)                AS payment_ts
FROM sales_bz.payments_bz;

Shipments: typing + standardization

In [0]:
%sql
CREATE OR REPLACE TABLE sales_sv.shipments_sv AS
SELECT
  CAST(shipment_id AS BIGINT)                    AS shipment_id,
  CAST(order_id AS BIGINT)                       AS order_id,
  CAST(ship_date AS TIMESTAMP)                   AS ship_ts,
  TRIM(carrier)                                  AS carrier,
  TRIM(service)                                  AS service,
  UPPER(TRIM(ship_status))                       AS ship_status,
  CAST(shipping_cost AS DECIMAL(18,2))           AS shipping_cost
FROM sales_bz.shipments_bz;

Refunds: typing

In [0]:
%sql
CREATE OR REPLACE TABLE sales_sv.refunds_sv AS
SELECT
  CAST(refund_id AS BIGINT)                      AS refund_id,
  CAST(order_id AS BIGINT)                       AS order_id,
  CAST(refund_amount AS DECIMAL(18,2))           AS refund_amount,
  TRIM(refund_reason)                            AS refund_reason,
  CAST(refund_date AS TIMESTAMP)                 AS refund_ts
FROM sales_bz.refunds_bz;